# Transfer data

Here, we'll show how to load one instance for reading data and use another to write data.

## Setup

In [ ]:
!lamin init --storage ./test-transfer --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb

lb.settings.organism = "human"
ln.settings.verbosity = "hint"

In [ ]:
ln.track()

## Query cellxgene

Query a dataset from the public `laminlabs/cellxgene` LaminDB instance that connects to data from the CZ CELLxGENE (for more info, see [here](docs:cellxgene)):

In [ ]:
# get a QuerySet containing all artifacts in cellxgene
# artifacts = ln.Artifact.using("laminlabs/cellxgene")
# artifacts.df().head()

Filter or search the queryset:

In [ ]:
# artifact = artifacts.filter(description__icontains="tabula sapiens").first()
# artifact

All associated features and labels:

In [ ]:
# artifact.describe()

The artifact id on the remote source database is: 

In [ ]:
# artifact.id

The artifact transform on the remote source database is:

In [ ]:
# artifact.transform

The artifact was created on the remote source database by this user:

In [ ]:
# artifact.created_by

In [ ]:
# artifact.storage

Save the dataset to our default instance:

In [ ]:
# artifact.save()

All features & labels have been transferred and now the artifact record points to the current database:

In [ ]:
# artifact.describe()

All ids and related attributes have been adjusted:

In [ ]:
# artifact.id

In [ ]:
# artifact.created_by

In [ ]:
# artifact.transform

And the database is populated correspondingly:

In [ ]:
ln.view()

In [ ]:
# clean up test instance
!lamin delete --force test-transfer
!rm -r test-transfer